In [1]:
cd "/Users/sylviathsu/Documents/COS243/LocalAILibrarian"

/Users/sylviathsu/Documents/COS243/LocalAILibrarian


In [2]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

In [3]:
import os
import logging
import gradio as gr
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

/opt/anaconda3/envs/gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
import logging
import gradio as gr
from pathlib import Path
from typing import List, Tuple

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    Settings
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def process_documents(doc_folder: str):
    """Process documents from the specified folder"""
    try:
        reader = SimpleDirectoryReader(
            input_dir=doc_folder,
            recursive=True,
            filename_as_id=True,
            required_exts=[".txt", ".epub", ".pdf"]
        )
        return reader.load_data()
    except Exception as e:
        logger.error(f"Document processing error: {e}")
        raise

def generate_embeddings(documents):
    """Generate embeddings and create vector store index"""
    try:
        # Configure embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        Settings.embed_model = embed_model
        
        # Configure text splitting
        splitter = SentenceSplitter(
            chunk_size=1024,
            chunk_overlap=200,
            paragraph_separator="\n\n",
            include_metadata=True
        )
        
        # Create and save index with transformations
        vector_index = VectorStoreIndex.from_documents(
            documents,
            transformations=[splitter],  # Apply transformations here
            embed_model=embed_model
        )
        
        storage_context = vector_index.storage_context
        storage_context.persist(persist_dir="storage")
        
        return vector_index
    except Exception as e:
        logger.error(f"Embedding generation error: {e}")
        raise

def configure_query_engine():
    """Configure the query engine with Ollama"""
    try:
        llm = Ollama(model="phi3.5:3.8b-mini-instruct-q4_K_M")
        Settings.llm = llm
        
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(storage_context)
        
        query_engine = index.as_query_engine(
            similarity_top_k=5,
            response_mode="tree_summarize",
            streaming=True,
            timeout=30
        )
        
        return query_engine
    except Exception as e:
        logger.error(f"Query engine configuration error: {e}")
        raise

def create_gradio_interface(query_engine):
    """Create and configure the Gradio interface"""
    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        try:
            response = query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Search your document collection using natural language queries.")
        
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
    return app

if __name__ == "__main__":
    try:
        # Step 1: Set up logging
        logger.info("Starting application...")
        
        # Step 2: Check if index exists
        if not Path("storage").exists():
            logger.info("Processing documents...")
            doc_folder = "./library"
            documents = process_documents(doc_folder)
            
            logger.info("Generating embeddings and saving index...")
            vector_index = generate_embeddings(documents)
        
        # Step 3: Configure query engine
        logger.info("Configuring query engine...")
        query_engine = configure_query_engine()
        
        # Step 4: Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface(query_engine)
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Configuring query engine...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:__main__:Launching Gradio interface...
INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET http://127.0.0.1:7865/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7865/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
